In [37]:
import os
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pymorphy2
import seaborn as sns
sns.set_style("whitegrid")
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
from string import punctuation

# HW1:  Сравнение стилей текстов
### Выполнили:  Булгаков Дмитрий, Тефикова Алие
### Группа ИАД-2

# 1. Загрузка данных

Составьте самостоятельно как минимум две коллекции
текстов разных стилей (например, коллекция текстов в публицистическом
стиле и коллекция текстов в научном стиле). Коллекции текстов
должны быть достаточно большие (порядка 5000 токенов). Посчитайте
количество токенов и типов в каждой коллекции.

### 1.1 Получение  данных из файла

In [38]:
def remove_control_characters(text_string):
    return ''.join(filter(None, text_string.splitlines()))

### 1.1.1 Загрузка художественных текстов (Портрет Дориана Грея,  Оскар Уайльд)

«Портре́т Дориана Гре́я» (англ. The Picture of Dorian Gray) — единственный опубликованный роман Оскара Уайльда. В жанровом отношении представляет смесь романа воспитания с моральной притчей. Существует в двух версиях — в 13 главах (1890 года) и в 20 главах (1891 года). Стал самым успешным произведением Уайльда, более 30 раз экранизировался.

<b>Link:<b> http://lib.ru/WILDE/doriangray.txt

In [39]:
fiction = open('data/dorian_gray.txt', encoding='utf-8').read()
fiction = remove_control_characters(fiction)

### 1.1.2 Загрузка  публицистических текстов (статьи lenta.ru) 

Lenta.ru — одно из ведущих российских новостных интернет-изданий, основанное в 1999 году Антоном Носиком при содействии Фонда эффективной политики. Работает круглосуточно, освещая мировые и внутрироссийские новости.

<b>Link<b>: http://lenta.ru

In [40]:
journalistic = open('data/lentaru.txt', encoding='utf-8').read()
journalistic = remove_control_characters(journalistic)

### 1.1.3 Загрузка  научных текстов (Молодежная Наука 2016)

Материалы Всероссийской научно-практической конференции молодых ученых, аспирантов и студентов, посвященной 150-летию со дня рождения профессора В.Н. Варгина (Пермь, 14-18 марта 2016 года)

<b>Link<b>: http://pgsha.ru/web/science/scientificarticles/

In [41]:
scientific = open('data/perm_conf.txt', encoding='utf-8').read()
scientific = remove_control_characters(scientific)

### 1.1.4 Загрузка  текстов  разговорного стиля (корпус составленный на базе Twitter )

В качестве источника текстов была выбрана платформа микроблогинга Twitter. Современные поисковые системы и имеющиеся в открытом доступе инструменты по сбору текстовых отзывов не позволяют собирать актуальные отзывы и оперативно работать с данными. В связи с этим на основе программного интерефейса API twitter  был разработан программный инструмент для извлечения отзывов об интересующих товарах, услугах,  событиях,  персонах из микроблоггинг-платформы twitter,  который позволяет учитывать время публикации сообщения и авторитетность автора сообщения. Этот инструмент использовался для сбора неразмеченного корпуса. В корпусе содержится более 15 миллионов записей за время с конца ноября 2013 года до конца февраля 2014 года.

<b>Link:<b> http://study.mokoron.com

In [42]:
conversational = open('data/twitter.txt', encoding='utf-8').read()
conversational = remove_control_characters(conversational)

### 1.2 Подсчет токенов и типов

In [43]:
exclude_symbols = set(punctuation + '0123456789'+u'–—'+u'«»'+u'“')

In [44]:
def tokenize(text, exlude_symb):
    text = text.lower()
    text_merged = ''.join(ch for ch in text if ch not in exlude_symb)
    text_tokens = WhitespaceTokenizer().tokenize(text_merged.lower())
    return text_tokens

In [45]:
def print_results(tokens):
    print('N of tokens: ', len(tokens))
    types = nltk.FreqDist(tokens)
    print('N of types:', len(types))
    print(types)

### 1.2.1 Токены и типы для  художественного стиля

In [46]:
fiction_tokens = tokenize(fiction, exclude_symbols)

In [47]:
for i in fiction_tokens[:10]: 
    print(i) 

художник
тот
кто
создает
прекрасное
раскрыть
людям
себя
и
скрыть


In [49]:
print_results(fiction_tokens)

N of tokens:  60528
N of types: 18236
<FreqDist with 18236 samples and 60528 outcomes>


### 1.2.2 Токены и типы для публицистического стиля

In [32]:
journalistic_tokens = tokenize(journalistic, exclude_symbols)

In [33]:
for i in journalistic_tokens[:10]: 
    print(i) 

на
борту
упавшего
в
телецкое
озеро
вертолета
robinson
находился
бывший


In [34]:
print_results(journalistic_tokens)

N of tokens:  13699
N of types: 6015
<FreqDist with 6015 samples and 13699 outcomes>


### 1.2.3 Токены и типы для  научного стиля

In [35]:
scientific_tokens = tokenize(scientific, exclude_symbols)

In [36]:
for i in scientific_tokens[:10]: 
    print(i) 

влияние
соотношения
компонентовна
урожайность
горохоячменной
смесианнотация
в
работе
представлены
результаты


In [88]:
print_results(scientific_tokens)

N of tokens:  32623
N of types: 5222
<FreqDist with 5222 samples and 32623 outcomes>


### 1.2.4 Токены и типы для  разговорного стиля

In [89]:
conversational_tokens = tokenize(conversational, exclude_symbols)

In [90]:
for i in conversational_tokens[:10]: 
    print(i) 

i
didnt
think
she
was
his
typeneither
did
iwhen
did


In [91]:
print_results(conversational_tokens)

N of tokens:  270145
N of types: 23778
<FreqDist with 23778 samples and 270145 outcomes>


### 2. Подсчет частей речи

Используя любой морфологический процессор, который вам нравится (pymorphy2, mystem), определите к какой части речиотносятся слова из каждой коллекции текстов. При помощи nltk.FreqDist() составьте частотные словари: часть речи – количество слов, к ней относящихся.

In [92]:
mystopwords = stopwords.words('english')

#### fiction

In [107]:
morph = pymorphy2.MorphAnalyzer()
lemmata = nltk.FreqDist()

mystopwords = stopwords.words('english')

f_types = nltk.FreqDist(fiction_tokens)

In [105]:
print(morph.parse('picture')[0].tag)

LATN


In [27]:
lemmata_no_sw = nltk.FreqDist()
for l in lemmata:
    if not l in mystopwords:
        lemmata_no_sw[l] = lemmata[l]
for i in lemmata_no_sw.most_common(20):
    print(i[0], i[1])

said 2463
one 1697
prince 1383
pierre 1214
would 1157
could 941
man 867
andrew 829
went 769
time 748
natásha 722
old 699
know 688
face 687
french 681
men 647
eyes 633
princess 630
thought 625
like 620


#### conversational

In [28]:
morph = pymorphy2.MorphAnalyzer()
lemmata = nltk.FreqDist()

mystopwords = stopwords.words('english')

c_types = nltk.FreqDist(conversational_tokens)
for t in c_types:
    try:
        l = morph.parse(t)[0].normal_form
        if l in lemmata:
            lemmata[l] += c_types[t]
        else:
            lemmata[l] = c_types[t]
    except IndexError:
        if t in lemmata:
            lemmata[t] += c_types[t]
        else:
            lemmata[t] = c_types[t]
print('N of lemmata:', len(lemmata))
for i in lemmata.most_common(10):
    print(i[0], i[1])

N of lemmata: 23778
you 8331
the 7514
a 6606
to 6151
i 4950
of 2836
your 2361
it 2304
in 2249
my 2232


In [29]:
lemmata_no_sw = nltk.FreqDist()
for l in lemmata:
    if not l in mystopwords:
        lemmata_no_sw[l] = lemmata[l]
for i in lemmata_no_sw.most_common(20):
    print(i[0], i[1])

get 1615
dont 1552
know 1358
got 1164
im 1139
like 1131
want 1065
youre 968
go 878
gonna 864
need 801
good 610
cant 590
think 585
take 578
come 558
really 508
one 504
right 501
going 494


#### scientific

In [30]:
morph = pymorphy2.MorphAnalyzer()
lemmata = nltk.FreqDist()

mystopwords = stopwords.words('english')

s_types = nltk.FreqDist(scientific_tokens)
for t in s_types:
    try:
        l = morph.parse(t)[0].normal_form
        if l in lemmata:
            lemmata[l] += s_types[t]
        else:
            lemmata[l] = s_types[t]
    except IndexError:
        if t in lemmata:
            lemmata[t] += s_types[t]
        else:
            lemmata[t] = s_types[t]
print('N of lemmata:', len(lemmata))
for i in lemmata.most_common(10):
    print(i[0], i[1])

N of lemmata: 5222
of 1441
the 1347
in 1114
and 1000
with 643
to 580
a 519
patients 476
is 456
for 319


In [31]:
lemmata_no_sw = nltk.FreqDist()
for l in lemmata:
    if not l in mystopwords:
        lemmata_no_sw[l] = lemmata[l]
for i in lemmata_no_sw.most_common(20):
    print(i[0], i[1])

patients 476
risk 250
disease 223
renal 210
ckd 167
cvd 163
may 161
heart 150
failure 145
dialysis 124
associated 122
esrd 119
levels 117
increased 116
factors 116
mortality 116
studies 107
also 103
kidney 96
cardiovascular 95


#### journalistic

In [32]:
morph = pymorphy2.MorphAnalyzer()
lemmata = nltk.FreqDist()

mystopwords = stopwords.words('english') + [u'mln', u'dlrs', u'vs', u'pct', u'cts', u'us', u'would']

j_types = nltk.FreqDist(journalistic_tokens)
for t in j_types:
    try:
        l = morph.parse(t)[0].normal_form
        if l in lemmata:
            lemmata[l] += j_types[t]
        else:
            lemmata[l] = j_types[t]
    except IndexError:
        if t in lemmata:
            lemmata[t] += j_types[t]
        else:
            lemmata[t] = j_types[t]
print('N of lemmata:', len(lemmata))
for i in lemmata.most_common(10):
    print(i[0], i[1])

N of lemmata: 34354
the 16142
of 8666
to 8377
said 6688
and 6345
a 6179
in 6140
mln 3658
for 3190
dlrs 2873


In [33]:
lemmata_no_sw = nltk.FreqDist()
for l in lemmata:
    if not l in mystopwords:
        lemmata_no_sw[l] = lemmata[l]
for i in lemmata_no_sw.most_common(20):
    print(i[0], i[1])

said 6688
year 1400
billion 1257
company 990
net 974
inc 842
loss 840
bank 765
new 741
corp 714
also 653
last 652
one 643
march 631
sales 601
share 593
stock 570
shares 556
market 553
profit 534
